In [0]:
# Creating the Title_df DataFrame from the bronze_title_ratings table
# removing null values from the table
title_df = spark.table('az_adb_simbus_training.imdb_project_schema.bronze_title_ratings').dropna(how='any')
# title_df.limit(5).display()

In [0]:
# Creating the basics_df DataFrame from the bronze_title_basics tables 
# selected only the columns required for the use case and removing null values from the selected columns
# selected the year from the startYear column and renamed it to Year
from pyspark.sql.functions import year, col
basics_df = spark.table('az_adb_simbus_training.imdb_project_schema.bronze_title_basics')
basics_df = basics_df.select('tconst','startYear','genres','titleType').dropna(how='any')
basics_df = basics_df.withColumn('startYear', year(col('startYear'))) \
                     .withColumnRenamed('startYear','Year')
# basics_df.limit(5).display()

In [0]:
# Performing an inner join between the title_df and title_ratings tables.
joined_df = title_df.join(basics_df,title_df.tconst == basics_df.tconst, "inner")
# joined_df.limit(5).display()

In [0]:
#Checking the distinct values of the titleType column
joined_df.select('titleType').distinct().show()

In [0]:
# Filter out only movies (titleType == 'movie'). 
# Extract year, genres, average_Rating, num_Votes. 
from pyspark.sql.functions import col 
selected_df = (joined_df.filter(col('titleType') == 'movie')               
                .select('Year','genres','averageRating', 'numVotes', 'titleType'))
selected_df = selected_df.dropna(how='any')                
# selected_df.limit(10).display()                           

In [0]:
# Filter out year is greater than 2000
# From genres column, extract only the first genre
from pyspark.sql.functions import split
Filtered_df = selected_df.filter(col('Year') >= 2000)
Filtered_df = Filtered_df.withColumn('genres', split(col('genres'),',')[0])
# Filtered_df.limit(5).display()


In [0]:
# Created the silver_UseCase1_table table in the imdb_project_schema schema from the above filtration.
Filtered_df.write.mode('overwrite').option('mergeSchema', 'true').saveAsTable('az_adb_simbus_training.imdb_project_schema.silver_UseCase1_table')


In [0]:
%sql
select * from az_adb_simbus_training.imdb_project_schema.silver_usecase1_table limit 5;
